# Using the `json` type

First you will need to [install](../../getting_started#installation) and [connect](../../getting_started#connection) to a DataJoint [data pipeline](https://datajoint.com/docs/glossary#data-pipeline).

Now let's start by importing the `datajoint` client.

In [1]:
import datajoint as dj


## Table Definition

For this exercise, let's imagine we work for an awesome company that is organizing a fun RC car race across various teams in the company. Let's see which team has the fastest car! 🏎️

This establishes 2 important entities: a `Team` and a `Car`. Normally we'd map this to their own dedicated table, however, let's assume that `Team` is well-structured but `Car` is less structured then we'd prefer. In other words, the structure for what makes up a *car* is varing too much between entries (perhaps because users of the pipeline haven't agreed yet on the definition? 🤷).

This would make it a good use-case to keep `Team` as a table but make `Car` actually a `json` type defined within the `Team` table.

Let's begin.

In [2]:
schema = dj.Schema(f"{dj.config['database.user']}_json")


[2023-02-08 03:47:29,554][INFO]: Connecting root@fakeservices.datajoint.io:3306
[2023-02-08 03:47:29,823][INFO]: Connected root@fakeservices.datajoint.io:3306


In [3]:
@schema
class Team(dj.Lookup):
    definition = """
    # A team within a company
    name: varchar(40)  # team name
    ---
    car=null: json  # A car belonging to a team (null to allow registering first but specifying car later)
    """


## Insert

Let's suppose that engineering is first up to register their car.

In [4]:
Team.insert1(
    {
        "name": "engineering",
        "car": {
            "name": "Rever",
            "length": 20.5,
            "inspected": True,
            "tire_pressure": [32, 31, 33, 34],
            "headlights": [
                {
                    "side": "left",
                    "hyper_white": None,
                },
                {
                    "side": "right",
                    "hyper_white": None,
                },
            ],
        },
    }
)


Next, business and marketing teams are up and register their cars.

A few points to notice below:
- The person signing up on behalf of marketing does not know the specifics of the car during registration but another team member will be updating this soon before the race.
- Notice how the `business` and `engineering` teams appear to specify the same property but refer to it as `safety_inspected` and `inspected` respectfully.

In [5]:
Team.insert(
    [
        {
            "name": "marketing",
            "car": None,
        },
        {
            "name": "business",
            "car": {
                "name": "Chaching",
                "length": 100,
                "safety_inspected": False,
                "tire_pressure": [34, 30, 27, 32],
                "headlights": [
                    {
                        "side": "left",
                        "hyper_white": True,
                    },
                    {
                        "side": "right",
                        "hyper_white": True,
                    },
                ],
            },
        },
    ]
)


We can preview the table data much like normal but notice how the value of `car` behaves like other BLOB-like attributes.

In [6]:
Team()

name team name,car A car belonging to a team (null to allow registering first but specifying car later)
business,=BLOB=
engineering,=BLOB=
marketing,=BLOB=


## Restriction

Now let's see what kinds of queries we can form to demostrate how we can query this pipeline.

In [7]:
# Which team has a `car` equal to 100 inches long?
Team & {'car.length': 100}

name team name,car A car belonging to a team (null to allow registering first but specifying car later)
business,=BLOB=


In [8]:
# Which team has a `car` less than 50 inches long?
Team & "car->>'$.length' < 50"

name team name,car A car belonging to a team (null to allow registering first but specifying car later)
engineering,=BLOB=


In [11]:
# Any team that has had their car inspected?
Team & [{'car.inspected:unsigned': True}, {'car.safety_inspected:unsigned': True}]

name team name,car A car belonging to a team (null to allow registering first but specifying car later)
engineering,=BLOB=


In [12]:
# Which teams do not have hyper white lights for their first head light?
Team & {"car.headlights[0].hyper_white": None}

name team name,car A car belonging to a team (null to allow registering first but specifying car later)
engineering,=BLOB=
marketing,=BLOB=


Notice that the previous query will satisfy the `None` check if it experiences any of the following scenarious:
- if entire record missing (`marketing` satisfies this)
- JSON key is missing
- JSON value is set to JSON `null` (`engineering` satisfies this)

## Projection?

Projections can be quite useful with the `json` type since we can extract out just what we need. This allows greater query flexibility.

In [30]:
# Only interested in the car names and the average tire pressure
Team.proj(
    car_name='car.name',
    car_avg_tire_pressure="AVG(car->>'$.tire_pressure')",
)

OperationalError: (1140, "In aggregated query without GROUP BY, expression #1 of SELECT list contains nonaggregated column 'root_json.#team.name'; this is incompatible with sql_mode=only_full_group_by")

OperationalError: (1140, "In aggregated query without GROUP BY, expression #1 of SELECT list contains nonaggregated column 'root_json.#team.name'; this is incompatible with sql_mode=only_full_group_by")

## Indices?

## Describe

Lastly, the `.describe()` function on the `Team` table can help us generate the table's definition. This is useful if we are connected directly to the pipeline without the original source.

In [20]:
definition = Team.describe()
print(definition)

# A team within a company
name                 : varchar(40)                  # team name
---
car=null             : json                         # A car belonging to a team (null to allow registering first but specifying car later)



## Cleanup

Finally, let's clean up what we created in this tutorial.

In [ ]:
schema.drop()
